In [5]:
# !pip install -q unsloth==0.6.5 accelerate>=0.30.1 bitsandbytes datasets peft transformers==4.41.1
# !pip install -q unsloth==0.6.5 'accelerate>=0.30.1' bitsandbytes datasets peft transformers==4.41.1
# !pip install -q unsloth==2025.10.1 'accelerate>=0.30.1' bitsandbytes datasets peft transformers==4.41.1
import os
# Remove '==4.41.1' from transformers to let Unsloth choose the right version
# !pip install -q unsloth==2025.10.1 'accelerate>=0.30.1' bitsandbytes datasets peft transformers

from datasets import load_dataset
from unsloth import FastLanguageModel, Trainer, TrainingArguments, SFTTrainer, get_peft_model
from transformers import AutoTokenizer

ImportError: cannot import name 'Trainer' from 'unsloth' (/home/pope/.local/lib/python3.10/site-packages/unsloth/__init__.py)

In [9]:
# Corrected imports
from unsloth import FastLanguageModel, SFTTrainer, get_peft_model
from transformers import AutoTokenizer, TrainingArguments

ImportError: cannot import name 'SFTTrainer' from 'unsloth' (/home/pope/.local/lib/python3.10/site-packages/unsloth/__init__.py)

In [ ]:
MODEL_NAME = "Qwen/Qwen1.5-7B-Chat"
OUTPUT_DIR = "outputs/qwen-finetuned"
DATA_PATH = "dataset_to train.json"  # update to actual file
TEXT_FIELD = "text"                       # update if column name differs
MAX_SEQ_LENGTH = 2048
BATCH_SIZE = 1
GRADIENT_ACCUMULATION = 16
LEARNING_RATE = 2e-5
NUM_EPOCHS = 3
WARMUP_RATIO = 0.03
SAVE_STEPS = 500
torch_dtype = "bfloat16"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
dataset = load_dataset("json", data_files=DATA_PATH)
train_dataset = dataset["train"]

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=torch_dtype,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    warmup_ratio=WARMUP_RATIO,
    fp16=False,
    bf16=torch_dtype == "bfloat16",
    logging_steps=10,
    save_steps=SAVE_STEPS,
    evaluation_strategy="no",
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="none",
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,
    dataset_text_field=TEXT_FIELD,
    max_seq_length=MAX_SEQ_LENGTH,
    args=training_args,
    packing=True,
)

trainer.train()


In [ ]:
model = FastLanguageModel.merge_lora(model, tokenizer)
model.save_pretrained(OUTPUT_DIR, safe_serialization=False)
tokenizer.save_pretrained(OUTPUT_DIR)

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=OUTPUT_DIR, max_new_tokens=256)
prompt = "### Instruction:\nExplain the benefits of knowledge graphs.\n\n### Response:"
print(pipe(prompt)[0]["generated_text"])